In [16]:
# PNC

import pickle
import re
import numpy as np

basis_file = '/home/anton/Documents/Tulane/Research/PNC_Good/AngleBasisNoJit20.pkl'
demodir = '/home/anton/Documents/Tulane/Research/ImageNomer/data/anton/cohorts/PNC/'

basis = pickle.load(open(basis_file, 'rb'))

thetas = []
jitter = []
age = []
sex = []
race = []
fc = []

demo = pickle.load(open(f'{demodir}/demographics.pkl', 'rb'))

for subtask in basis:
    m = re.search('([^-]+)-(.+)', subtask)
    sub = m.group(1)
    task = m.group(2)
    if sub not in demo['Race'] or demo['Race'][sub] not in ['AA', 'EA']:
        continue
    a = demo['age_at_cnb'][sub]
    s = demo['Sex'][sub] == 'M'
    r = demo['Race'][sub] == 'AA'
    age.append(a)
    sex.append(s)
    race.append(r)
    thetas.append(basis[subtask]['thetas'])
    jitter.append(basis[subtask]['jitter'])
    p = np.load(f'{demodir}/fc/{sub}_task-{task}_fc.npy')
    fc.append(p)
    
thetas = np.stack(thetas)
jitter = np.stack(jitter)
age = np.array(age).astype('int')
sex = np.array(sex).astype('int')
race = np.array(race).astype('int')
fc = np.stack(fc)

print([a.shape for a in [thetas, jitter, sex, race, age, fc]])
print(np.mean(1-sex))
print(np.mean(1-race))
print(np.mean(age))

[(3849, 20, 264), (3849, 20, 264), (3849,), (3849,), (3849,), (3849, 34716)]
0.5263704858404781
0.5188360613146272
14.398285268901013


In [4]:
import matplotlib.pyplot as plt

def mat2vec(mat):
    a,b = np.triu_indices(264,1)
    return mat[:,a,b]

a,b = np.triu_indices(264,1)
X = np.ones((fc.shape[0],264,264))
X[:,a,b] = fc
X[:,b,a] = fc
w, v = np.linalg.eig(X)
print('Done eigs')

w[:,20:] = 0
aps20 = np.real(np.einsum('nab,nb,ncb->nac',v,w,v))
aps20 = mat2vec(aps20)
print('Done 20')

w[:,15:] = 0
aps15 = np.real(np.einsum('nab,nb,ncb->nac',v,w,v))
aps15 = mat2vec(aps15)
print('Done 15')

w[:,10:] = 0
aps10 = np.real(np.einsum('nab,nb,ncb->nac',v,w,v))
aps10 = mat2vec(aps10)
print('Done 10')

w[:,5:] = 0
aps5 = np.real(np.einsum('nab,nb,ncb->nac',v,w,v))
aps5 = mat2vec(aps5)
print('Done 5')

w[:,3:] = 0
aps3 = np.real(np.einsum('nab,nb,ncb->nac',v,w,v))
aps3 = mat2vec(aps3)
print('Done 3')

w[:,1:] = 0
aps1 = np.real(np.einsum('nab,nb,ncb->nac',v,w,v))
aps1 = mat2vec(aps1)
print('Done 1')

w = None
v = None
X = None

Done eigs
Done 20
Done 15
Done 10
Done 5
Done 3
Done 1


In [17]:
def rmse(yhat, y):
    if isinstance(yhat, np.ndarray) or isinstance(yhat, int):
        f = np.mean
    else:
        f = torch.mean
    return f((y-yhat)**2)**0.5

def tops(thetas, jitter):
    t0 = np.expand_dims(thetas, 2)
    t1 = np.expand_dims(thetas, 3)
#     j0 = np.expand_dims(jitter, 2)
#     j1 = np.expand_dims(jitter, 3)
    ps = np.cos(t0-t1)#*(j0*j1)
    a,b = np.triu_indices(264, 1)
    ps = ps[:,:,a,b]
    return ps

paps = []
pres = []

for i in range(0,3849,500):
    ps = tops(thetas[i:i+500], jitter[i:i+500])
    aps = np.mean(ps, axis=1)
    res = fc[i:i+500] - aps
    paps.append(aps)
    pres.append(res)
    print(f'Done {i}')
    
aps = np.concatenate(paps)
res = np.concatenate(pres)

print(ps.shape)
print(aps.shape)
print(res.shape)

Done 0
Done 500
Done 1000
Done 1500
Done 2000
Done 2500
Done 3000
Done 3500
(349, 20, 34716)
(3849, 34716)
(3849, 34716)


In [18]:
aps20 = aps
res20 = res

print('Done')

Done


In [19]:
import sys

pth = '../../LatentSimilarity/'

if pth not in sys.path:
    sys.path.append(pth)
    
print(sys.path)

['/home/anton/Documents/Tulane/Research/code/AngleBasis', '/usr/lib/python310.zip', '/usr/lib/python3.10', '/usr/lib/python3.10/lib-dynload', '', '/home/anton/.local/lib/python3.10/site-packages', '/usr/lib/python3.10/site-packages', '../../LatentSimilarity/']


In [20]:
paps = None
pres = None

In [11]:
x0tr, x0t, x1tr, x1t, x2tr, x2t, x3tr, x3t, x4tr, x4t, x5tr, x5t, x6tr, x6t, ytr, yt = 16*[None]

In [24]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge, LogisticRegression
from sklearn.metrics import roc_auc_score
from latsim import LatSimClf
import warnings
warnings.filterwarnings("ignore")

def cat(x, **kwargs):
    return np.concatenate(x, **kwargs)

def rmse(yhat, y):
    if isinstance(yhat, np.ndarray) or isinstance(yhat, int):
        f = np.mean
    else:
        f = torch.mean
    return f((y-yhat)**2)**0.5

def predict(xtr, xt, ytr, yt, lst):
#     reg = LogisticRegression(C=1,max_iter=100).fit(xtr, ytr) 
    reg = LatSimClf(nepochs=1000).fit(xtr, ytr) 
#     reg = Ridge(alpha=1,max_iter=1000).fit(xtr, ytr)
    yhat = reg.predict(xt)
    p = reg.predict_proba(xt)
#     acc = rmse(yt, yhat)
#     acc = np.mean(yt == yhat)
    acc = roc_auc_score(yt, p[:,1])
    print(acc)
    lst.append(acc)
    return p
    
def get_res(fctr, fct, abtr, abt):
    return fctr-abtr, fct-abt

def combine(yt, p0, p1, lst):
    p = (p0+p1)/2
#     acc = np.mean(yt == np.argmax(p, axis=1))
    acc = roc_auc_score(yt, p[:,1])
#     acc = rmse(yt, p)
    print(acc)
    lst.append(acc)

rfc = []

rab1 = []
rab3 = []
rab5 = []
rab10 = []
rab15 = []
rab20 = []

rres1 = []
rres3 = []
rres5 = []
rres10 = []
rres15 = []
rres20 = []

rens1 = []
rens3 = []
rens5 = []
rens10 = []
rens15 = []
rens20 = []

rbest = []

for i in range(10):

#     xtr, xt, ytr, yt = train_test_split(fc, sex, stratify=sex, train_size=0.8)
    
#     p1 = predict(xtr, xt, ytr, yt, rfc)
#     p2 = predict(xtr, xt, ytr, yt, rfc)
    
#     combine(yt, p1, p2, rbest)

    x0tr, x0t, x1tr, x1t, x2tr, x2t, x3tr, x3t, x4tr, x4t, x5tr, x5t, x6tr, x6t, ytr, yt = train_test_split(
        fc, aps1, aps3, aps5, aps10, aps15, aps20, race, stratify=race, train_size=0.8)
    
#     mu = np.mean(ytr)
#     ytr = ytr-mu
#     yt = yt-mu
    
    x1atr, x1at = get_res(x0tr, x0t, x1tr, x1t)
    x2atr, x2at = get_res(x0tr, x0t, x2tr, x2t)
    x3atr, x3at = get_res(x0tr, x0t, x3tr, x3t)
    x4atr, x4at = get_res(x0tr, x0t, x4tr, x4t)
    x5atr, x5at = get_res(x0tr, x0t, x5tr, x5t)
    x6atr, x6at = get_res(x0tr, x0t, x6tr, x6t)

    predict(x0tr, x0t, ytr, yt, rfc)
    
    p1 = predict(x1tr, x1t, ytr, yt, rab1)
    p2 = predict(x2tr, x2t, ytr, yt, rab3)
    p3 = predict(x3tr, x3t, ytr, yt, rab5)
    p4 = predict(x4tr, x4t, ytr, yt, rab10)
    p5 = predict(x5tr, x5t, ytr, yt, rab15)
    p6 = predict(x6tr, x6t, ytr, yt, rab20)
    
    p1a = predict(x1atr, x1at, ytr, yt, rres1)
    p2a = predict(x2atr, x2at, ytr, yt, rres3)
    p3a = predict(x3atr, x3at, ytr, yt, rres5)
    p4a = predict(x4atr, x4at, ytr, yt, rres10)
    p5a = predict(x5atr, x5at, ytr, yt, rres15)
    p6a = predict(x6atr, x6at, ytr, yt, rres20)
    
    combine(yt, p1, p1a, rens1)
    combine(yt, p2, p2a, rens3)
    combine(yt, p3, p3a, rens5)
    combine(yt, p4, p4a, rens10)
    combine(yt, p5, p5a, rens15)
    combine(yt, p6, p6a, rens20)

    combine(yt, p6, p1a, rbest)
    
    print('---')
    
print(np.mean(rfc), np.std(rfc))
print(np.mean(rbest), np.std(rbest))

print(np.mean(rab1), np.std(rab1))
print(np.mean(rab3), np.std(rab3))
print(np.mean(rab5), np.std(rab5))
print(np.mean(rab10), np.std(rab10))
print(np.mean(rab15), np.std(rab15))
print(np.mean(rab20), np.std(rab20))

print(np.mean(rres1), np.std(rres1))
print(np.mean(rres3), np.std(rres3))
print(np.mean(rres5), np.std(rres5))
print(np.mean(rres10), np.std(rres10))
print(np.mean(rres15), np.std(rres15))
print(np.mean(rres20), np.std(rres20))

print(np.mean(rens1), np.std(rens1))
print(np.mean(rens3), np.std(rens3))
print(np.mean(rens5), np.std(rens5))
print(np.mean(rens10), np.std(rens10))
print(np.mean(rens15), np.std(rens15))
print(np.mean(rens20), np.std(rens20))

0.9518006756756756
0.7958479729729728
0.9164425675675675
0.9388716216216216
0.9417871621621622
0.9510709459459459
0.9467094594594596
0.7396891891891891
0.8116216216216217
0.8612263513513512
0.8610844594594596
0.809222972972973
0.7555945945945945
0.7855168918918919
0.889081081081081
0.9245101351351351
0.9373716216216217
0.9359932432432434
0.9211351351351352
0.9005371621621622
---
0.9458513513513513
0.8415777027027028
0.9128581081081081
0.9422364864864865
0.9544493243243243
0.9431824324324325
0.9422972972972973
0.7827263513513514
0.8337094594594594
0.8545405405405405
0.8694155405405405
0.8230641891891892
0.7561587837837838
0.8346858108108108
0.8933918918918918
0.9292635135135136
0.9485608108108109
0.9344797297297298
0.9157905405405405
0.9101587837837839
---
0.9397229729729729
0.8142601351351351
0.9170236486486485
0.925347972972973
0.9317364864864865
0.9411081081081081
0.9362466216216218
0.7325472972972973
0.8485743243243243
0.8771925675675676
0.8585945945945946
0.8009797297297298
0.76716